# Movie Analysis for Microsoft's New Movie Studio

## Overview
This project analyzes various movie datasets to provide actionable insights for Microsoft's new movie studio. The goal is to identify key factors that contribute to a film's success, helping Microsoft make data-driven decisions on what types of films to produce.

## Business Understanding
**Stakeholder:** Microsoft's new movie studio.

**Business Problem:** Microsoft wants to enter the movie production market but lacks experience in creating films. They need insights into the types of movies that are currently performing well at the box office to inform their production strategy.

**Key Business Questions:**
1. What genres are most profitable?
2. Which movie attributes correlate with higher box office returns?
3. How do critical and audience ratings impact a movie's financial success?

## Data Understanding
The analysis uses data from multiple sources:
- **Box Office Mojo (bom.movie_gross.csv):** Contains box office data.
- **IMDB (im.db):** SQLite database with detailed movie information and ratings.
- **The Movie Database (tmdb.movies.csv):** Contains vote information.
- **The Numbers (tn.movie_budgets.csv):** Contains information about production budget, and gross revenue information.